# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [19]:
# import libraries
from sqlalchemy import create_engine
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report,confusion_matrix, precision_score,\
recall_score,accuracy_score,  f1_score,  make_scorer
from sklearn.base import BaseEstimator, TransformerMixin
import nltk

from nltk import word_tokenize

import pickle

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sqlalchemy import create_engine
import sqlite3



from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier



import pickle


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hamza\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hamza\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load daata from database
conn = sqlite3.connect('Clean_Messages.db')
df = pd.read_sql('SELECT * FROM Clean_Messages', conn)
df = df.dropna()
X = df["message"]
Y = df.drop("message",1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
     """
    Takes a Python string object and returns a list of processed words 
    of the text.
    
    INPUT:
        - text - Python str object - A raw text data
        
    OUTPUT:
        - stem_words - Python list object - A list of processed words from the input `text`.
    """
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [4]:
#remove all non numeric columns from the Y set 
Y = Y.drop("id",1)
Y = Y.drop("genre",1)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier(n_estimators=1000, random_state=0)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)
y_pred    

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [9]:
y_pred.shape

(8649, 36)

In [25]:
category_names=Y.columns

In [154]:
y_test

,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
5787,direct,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9736,direct,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3350,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5394,direct,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18125,news,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4828,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12491,social,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
5027,direct,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18016,news,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
x=0
for column in y_test.columns:
    
    print(classification_report(y_test[column], y_pred[:,x]))
    x=x+1

              precision    recall  f1-score   support

         0.0       0.38      0.02      0.03      2042
         1.0       0.76      0.99      0.86      6546
         2.0       0.00      0.00      0.00        61

    accuracy                           0.75      8649
   macro avg       0.38      0.34      0.30      8649
weighted avg       0.66      0.75      0.66      8649

              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91      7194
         1.0       0.47      0.02      0.04      1455

    accuracy                           0.83      8649
   macro avg       0.65      0.51      0.47      8649
weighted avg       0.77      0.83      0.76      8649

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      8615
         1.0       0.00      0.00      0.00        34

    accuracy                           1.00      8649
   macro avg       0.50      0.50      0.50      8649
weighted avg       0

C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      8513
         1.0       0.00      0.00      0.00       136

    accuracy                           0.98      8649
   macro avg       0.49      0.50      0.50      8649
weighted avg       0.97      0.98      0.98      8649

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      8452
         1.0       0.17      0.01      0.01       197

    accuracy                           0.98      8649
   macro avg       0.57      0.50      0.50      8649
weighted avg       0.96      0.98      0.97      8649



C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      8544
         1.0       0.00      0.00      0.00       105

    accuracy                           0.99      8649
   macro avg       0.49      0.50      0.50      8649
weighted avg       0.98      0.99      0.98      8649

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      8319
         1.0       0.00      0.00      0.00       330

    accuracy                           0.96      8649
   macro avg       0.48      0.50      0.49      8649
weighted avg       0.93      0.96      0.94      8649

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98      8255
         1.0       0.00      0.00      0.00       394

    accuracy                           0.95      8649
   macro avg       0.48      0.50      0.49      8649
weighted avg       0.91      0.95      0.93      8649

              preci

C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      8473
         1.0       0.00      0.00      0.00       176

    accuracy                           0.98      8649
   macro avg       0.49      0.50      0.49      8649
weighted avg       0.96      0.98      0.97      8649

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      8603
         1.0       0.00      0.00      0.00        46

    accuracy                           0.99      8649
   macro avg       0.50      0.50      0.50      8649
weighted avg       0.99      0.99      0.99      8649



C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      8553
         1.0       0.00      0.00      0.00        96

    accuracy                           0.99      8649
   macro avg       0.49      0.50      0.50      8649
weighted avg       0.98      0.99      0.98      8649

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      8607
         1.0       0.00      0.00      0.00        42

    accuracy                           1.00      8649
   macro avg       0.50      0.50      0.50      8649
weighted avg       0.99      1.00      0.99      8649

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      8542
         1.0       0.00      0.00      0.00       107

    accuracy                           0.99      8649
   macro avg       0.49      0.50      0.50      8649
weighted avg       0.98      0.99      0.98      8649

              preci

In [26]:
metrics_list_all=[]
for col in range(y_test.shape[1]):
    accuracy = accuracy_score(y_test.iloc[:,col], y_pred[:,col])
    precision=precision_score(y_test.iloc[:,col], y_pred[:,col],average='micro')
    recall = recall_score(y_test.iloc[:,col], y_pred[:,col],average='micro')
    f_1 = f1_score(y_test.iloc[:,col], y_pred[:,col],average='micro')
    metrics_list=[accuracy,precision,recall,f_1]
    metrics_list_all.append(metrics_list)
metrics_df=pd.DataFrame(metrics_list_all,index=category_names,columns=["Accuracy","Precision","Recall","F_1"])
print(metrics_df)

                        Accuracy  Precision    Recall       F_1
related                 0.753266   0.753266  0.753266  0.753266
request                 0.831310   0.831310  0.831310  0.831310
offer                   0.995953   0.995953  0.995953  0.995953
aid_related             0.588276   0.588276  0.588276  0.588276
medical_help            0.921956   0.921956  0.921956  0.921956
medical_products        0.952133   0.952133  0.952133  0.952133
search_and_rescue       0.972714   0.972714  0.972714  0.972714
security                0.980229   0.980229  0.980229  0.980229
military                0.967395   0.967395  0.967395  0.967395
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.938259   0.938259  0.938259  0.938259
food                    0.894323   0.894323  0.894323  0.894323
shelter                 0.907041   0.907041  0.907041  0.907041
clothing                0.984276   0.984276  0.984276  0.984276
money                   0.976760   0.976

In [42]:
def avg_accuracy_score(y_true, y_pred):
    """
    Assumes that the numpy arrays `y_true` and `y_pred` ararys 
    are of the same shape and returns the average of the 
    accuracy score computed columnwise. 
    
    y_true - Numpy array - An (m x n) matrix 
    y_pred - Numpy array - An (m x n) matrix 
    
    avg_accuracy - Numpy float64 object - Average of accuracy score
    """
    
    # initialise an empty list
    accuracy_results = []
    
    # for each column index in either y_true or y_pred
    for idx in range(y_true.shape[-1]):
        # Get the accuracy score of the idx-th column of y_true and y_pred
        accuracy = accuracy_score(y_true[:,idx], y_pred[:,idx])
        
        # Update accuracy_results with accuracy
        accuracy_results.append(accuracy)
        
    # Take the mean of accuracy_results
    avg_accuracy = np.mean(accuracy_results)
    
    return avg_accuracy

In [45]:
average_accuracy_score =make_scorer(avg_accuracy_score)

In [28]:
list(pipeline.get_params())


['memory',
 'steps',
 'verbose',
 'vect',
 'tfidf',
 'clf',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__strip_accents',
 'vect__token_pattern',
 'vect__tokenizer',
 'vect__vocabulary',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'clf__estimator__bootstrap',
 'clf__estimator__ccp_alpha',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__max_samples',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'c

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = [
    {
        'clf__estimator__max_leaf_nodes': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4],
    }
]

cv = GridSearchCV(pipeline, param_grid=parameters,
                 scoring=average_accuracy_score, 
                verbose=10, 
                return_train_score=True 
                 )
cv.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START clf__estimator__max_leaf_nodes=50, clf__estimator__min_samples_split=2


C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "<ipython-input-42-9ea5b31b8221>", line 19, in avg_accuracy_score
    accuracy = accuracy_score(y_true[:,idx], y_pred[:,idx])
  File "C:\Users\Hamza\miniconda3\lib\site-packages\pandas\core\frame.py", line 3024, in __getitem__
    indexer = self.columns.ge

[CV 1/5; 1/9] END clf__estimator__max_leaf_nodes=50, clf__estimator__min_samples_split=2; total time=13.8min
[CV 2/5; 1/9] START clf__estimator__max_leaf_nodes=50, clf__estimator__min_samples_split=2


C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "<ipython-input-42-9ea5b31b8221>", line 19, in avg_accuracy_score
    accuracy = accuracy_score(y_true[:,idx], y_pred[:,idx])
  File "C:\Users\Hamza\miniconda3\lib\site-packages\pandas\core\frame.py", line 3024, in __getitem__
    indexer = self.columns.ge

[CV 2/5; 1/9] END clf__estimator__max_leaf_nodes=50, clf__estimator__min_samples_split=2; total time=13.5min
[CV 3/5; 1/9] START clf__estimator__max_leaf_nodes=50, clf__estimator__min_samples_split=2


...

[CV 5/5; 6/9] END clf__estimator__max_leaf_nodes=100, clf__estimator__min_samples_split=4; total time=17.9min
[CV 1/5; 7/9] START clf__estimator__max_leaf_nodes=200, clf__estimator__min_samples_split=2


C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "<ipython-input-42-9ea5b31b8221>", line 19, in avg_accuracy_score
    accuracy = accuracy_score(y_true[:,idx], y_pred[:,idx])
  File "C:\Users\Hamza\miniconda3\lib\site-packages\pandas\core\frame.py", line 3024, in __getitem__
    indexer = self.columns.ge

[CV 1/5; 7/9] END clf__estimator__max_leaf_nodes=200, clf__estimator__min_samples_split=2; total time=22.3min
[CV 2/5; 7/9] START clf__estimator__max_leaf_nodes=200, clf__estimator__min_samples_split=2


C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\Hamza\miniconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "<ipython-input-42-9ea5b31b8221>", line 19, in avg_accuracy_score
    accuracy = accuracy_score(y_true[:,idx], y_pred[:,idx])
  File "C:\Users\Hamza\miniconda3\lib\site-packages\pandas\core\frame.py", line 3024, in __getitem__
    indexer = self.columns.ge

[CV 2/5; 7/9] END clf__estimator__max_leaf_nodes=200, clf__estimator__min_samples_split=2; total time=24.1min
[CV 3/5; 7/9] START clf__estimator__max_leaf_nodes=200, clf__estimator__min_samples_split=2


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
metrics_list_all=[]
for col in range(y_test.shape[1]):
    accuracy = accuracy_score(y_test.iloc[:,col], y_pred[:,col])
    precision=precision_score(y_test.iloc[:,col], y_pred[:,col],average='micro')
    recall = recall_score(y_test.iloc[:,col], y_pred[:,col],average='micro')
    f_1 = f1_score(y_test.iloc[:,col], y_pred[:,col],average='micro')
    metrics_list=[accuracy,precision,recall,f_1]
    metrics_list_all.append(metrics_list)
metrics_df=pd.DataFrame(metrics_list_all,index=category_names,columns=["Accuracy","Precision","Recall","F_1"])
print(metrics_df)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [29]:
Y.

0        Un front froid se retrouve sur Cuba ce matin. ...
1                       Cyclone nan fini osinon li pa fini
2        Patnm, di Maryani relem pou li banm nouvel li ...
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        facade ouest d Haiti et le reste du pays aujou...
5                      Informtion au nivaux palais nationl
6                              Cyclone Coeur sacr de jesus
7        Tanpri nou bezwen tant avek dlo nou zon silo m...
8                    Mwen ta renmen jouin messag yo. Merci
9        Nou kwadebouke, nou gen pwoblem sant m yo nan ...
10                 Bon repo pa gen anyen menm grangou swaf
11                  M nan pv mvle plis enfomasyon sou 4636
12       Mwen thomassin 32 nan pyron mwen ta renmen jwe...
13       Ann fel ansanm bezwen manje nan delma 75 nan r...
14       gen plis enfomasyon sou 4636 la pom w sim ap k...
15       Komite katye delma 19 rue janvier imp charite ...
16       Nou bezwen mange avek dlo nan klcin 12 LA LAFI.

In [28]:
np.sum(Y.isnull())

id                            0
original                  16122
genre                         0
related                     138
request                     138
offer                       138
aid_related                 138
medical_help                138
medical_products            138
search_and_rescue           138
security                    138
military                    138
child_alone                 138
water                       138
food                        138
shelter                     138
clothing                    138
money                       138
missing_people              138
refugees                    138
death                       138
other_aid                   138
infrastructure_related      138
transport                   138
buildings                   138
electricity                 138
tools                       138
hospitals                   138
shops                       138
aid_centers                 138
other_infrastructure        138
weather_